# **Problem 1**

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#download the source
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')

#create a new dataframe
column_names = ['Postalcode','Borough','Neighborhood']
data = pd.DataFrame(columns = column_names)
data

,Postalcode,Borough,Neighborhood


In [3]:
for tr in table.find_all('tr'):
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    if len(row)==3:
        data.loc[len(data)] = row

In [4]:
data.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [5]:
#clean the data
not_assigned = data[data['Borough'] == 'Not assigned'].index
data.drop(not_assigned , inplace = True)

In [6]:
#as the neiborhood column is already combined, replace the slash with a comma
data['Neighborhood'] = data['Neighborhood'].str.replace(' / ',', ')

In [7]:
data.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
data.shape

(103, 3)

# **Problem 2**

In [9]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return lat_lng_coords

In [10]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lat_lng = pd.read_csv('Toronto_long_lat_data.csv')
df_lat_lng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_lat_lng.rename(columns={'Postal Code':'Postalcode'},inplace=True)
data1 = pd.merge(df_lat_lng, data, on='Postalcode')
geo_data = data1[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# **Problem 3**

In [12]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

Packages installed.


In [13]:
import geocoder # import geocoder
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
import numpy as np
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

print('Libraries imported.')

Libraries imported.


In [14]:
address = 'York'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [15]:
york_data=geo_data[geo_data['Borough'].str.contains("York")]
york_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
19,M2K,North York,Bayview Village,43.786947,-79.385975
20,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
21,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493


In [16]:
# @hiddencell
client_id = 'I1QQKNVV4PQXBZXEU2CHBLZIMLD4LBTBODR12CEGF23ASPMY'
client_secret = 'FORSGYRRFIEKBNRRPJ0YT4BIQ14BOEZ2222U52P0KGYKFWET'
version = '20200410'

In [17]:
def getNearbyVenues(names, latitudes, longitudes):
    radius = 500
    limit = 100
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
york_venues = getNearbyVenues(names = york_data['Neighborhood'], latitudes = york_data['Latitude'], longitudes = york_data['Longitude'])

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
North Park, Maple Leaf Park, Upwood Park
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Runnymede, The Junction North
Humber Summit
Humberlea, Emery
Weston


In [19]:
york_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


In [20]:
york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Caledonia-Fairbanks,4,4,4,4,4,4
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",4,4,4,4,4,4
Don Mills,26,26,26,26,26,26
Downsview,15,15,15,15,15,15
East Toronto,4,4,4,4,4,4
"Fairview, Henry Farm, Oriole",64,64,64,64,64,64


In [21]:
york_venues.shape

(335, 7)

In [22]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 118 uniques categories.


In [23]:
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

york_onehot['Neighborhood'] = york_venues['Neighborhood'] 

fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot.head(10)

,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,...,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hillcrest Village
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hillcrest Village
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hillcrest Village
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hillcrest Village
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hillcrest Village
5,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,"Fairview, Henry Farm, Oriole"
6,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,"Fairview, Henry Farm, Oriole"
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Fairview, Henry Farm, Oriole"
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Fairview, Henry Farm, Oriole"
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Fairview, Henry Farm, Oriole"


In [24]:
york_onehot.shape

(335, 119)

In [25]:
york_grouped = york_onehot.groupby('Neighborhood').mean().reset_index()

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = york_grouped['Neighborhood']

for ind in np.arange(york_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)
    
venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Diner,Shopping Mall,Pizza Place,Pharmacy,Supermarket,Sandwich Place,Sushi Restaurant,Ice Cream Shop
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Electronics Store,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Indian Restaurant,Liquor Store,Café,Butcher,Pharmacy,Pizza Place
3,Caledonia-Fairbanks,Park,Market,Women's Store,Fried Chicken Joint,Diner,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Discount Store,Restaurant,Sandwich Place,Coffee Shop,Yoga Studio,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice


In [27]:
# set number of clusters
kclusters = 5
york_grouped_clustering = york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 1, 0, 0, 4, 4, 0, 4, 0, 0, 3, 0, 0, 0, 4, 0, 0, 1, 0, 0,
       0, 1, 0, 2, 4, 4], dtype=int32)

In [28]:
# add clustering labels
venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

# merge to add latitude/longitude for each neighborhood
york_merged = york_data.join(venues_sorted.set_index('Neighborhood'), on='Neighborhood')

york_merged.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Dog Run,Golf Course,Pool,Mediterranean Restaurant,Fast Food Restaurant,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Restaurant,Tea Room,Women's Store,Food Court,Accessories Store,Bakery
19,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Japanese Restaurant,Chinese Restaurant,Café,Bank,Electronics Store,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega
20,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,2.0,Gym,Yoga Studio,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store
22,M2N,North York,Willowdale,43.770120,-79.408493,0.0,Coffee Shop,Ramen Restaurant,Pizza Place,Restaurant,Café,Discount Store,Sandwich Place,Sushi Restaurant,Pet Store,Bubble Tea Shop
23,M2P,North York,York Mills West,43.752758,-79.400049,4.0,Electronics Store,Bank,Convenience Store,Park,Food Truck,Food Court,Concert Hall,Construction & Landscaping,Furniture / Home Store,Cosmetics Shop
24,M2R,North York,Willowdale,43.782736,-79.442259,0.0,Coffee Shop,Ramen Restaurant,Pizza Place,Restaurant,Café,Discount Store,Sandwich Place,Sushi Restaurant,Pet Store,Bubble Tea Shop
25,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega
26,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Beer Store,Restaurant,Coffee Shop,Japanese Restaurant,Gym,Supermarket,Discount Store,Sandwich Place,Shopping Mall,Café


In [29]:
york_merged = york_merged.dropna()
york_merged['Cluster_Labels'] = york_merged.Cluster_Labels.astype(int)

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start = 14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighborhood'], york_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
#examine
york_merged.loc[york_merged['Cluster_Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,0,Dog Run,Golf Course,Pool,Mediterranean Restaurant,Fast Food Restaurant,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
18,North York,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Restaurant,Tea Room,Women's Store,Food Court,Accessories Store,Bakery
19,North York,0,Japanese Restaurant,Chinese Restaurant,Café,Bank,Electronics Store,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega
22,North York,0,Coffee Shop,Ramen Restaurant,Pizza Place,Restaurant,Café,Discount Store,Sandwich Place,Sushi Restaurant,Pet Store,Bubble Tea Shop
24,North York,0,Coffee Shop,Ramen Restaurant,Pizza Place,Restaurant,Café,Discount Store,Sandwich Place,Sushi Restaurant,Pet Store,Bubble Tea Shop
26,North York,0,Beer Store,Restaurant,Coffee Shop,Japanese Restaurant,Gym,Supermarket,Discount Store,Sandwich Place,Shopping Mall,Café
27,North York,0,Beer Store,Restaurant,Coffee Shop,Japanese Restaurant,Gym,Supermarket,Discount Store,Sandwich Place,Shopping Mall,Café
28,North York,0,Coffee Shop,Bank,Diner,Shopping Mall,Pizza Place,Pharmacy,Supermarket,Sandwich Place,Sushi Restaurant,Ice Cream Shop
29,North York,0,Coffee Shop,Miscellaneous Shop,Caribbean Restaurant,Massage Studio,Bar,Falafel Restaurant,Yoga Studio,Discount Store,Cosmetics Shop,Curling Ice
34,North York,0,Coffee Shop,Portuguese Restaurant,Hockey Arena,Financial or Legal Service,Intersection,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice


In [32]:
york_merged.loc[york_merged['Cluster_Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,North York,1,Park,Food & Drink Shop,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega
74,York,1,Park,Market,Women's Store,Fried Chicken Joint,Diner,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
98,York,1,Park,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store


In [33]:
york_merged.loc[york_merged['Cluster_Labels'] == 2, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,North York,2,Gym,Yoga Studio,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store


In [34]:
york_merged.loc[york_merged['Cluster_Labels'] == 3, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,North York,3,Baseball Field,Yoga Studio,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store


In [35]:
york_merged.loc[york_merged['Cluster_Labels'] == 4, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,4,Electronics Store,Bank,Convenience Store,Park,Food Truck,Food Court,Concert Hall,Construction & Landscaping,Furniture / Home Store,Cosmetics Shop
30,North York,4,Park,Baseball Field,Grocery Store,Gym / Fitness Center,Home Service,Hotel,Discount Store,Korean Restaurant,Liquor Store,Shopping Mall
31,North York,4,Park,Baseball Field,Grocery Store,Gym / Fitness Center,Home Service,Hotel,Discount Store,Korean Restaurant,Liquor Store,Shopping Mall
32,North York,4,Park,Baseball Field,Grocery Store,Gym / Fitness Center,Home Service,Hotel,Discount Store,Korean Restaurant,Liquor Store,Shopping Mall
33,North York,4,Park,Baseball Field,Grocery Store,Gym / Fitness Center,Home Service,Hotel,Discount Store,Korean Restaurant,Liquor Store,Shopping Mall
36,East York,4,Skating Rink,Park,Pharmacy,Cosmetics Shop,Curling Ice,Dance Studio,Beer Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
40,East York,4,Coffee Shop,Convenience Store,Intersection,Park,Dog Run,Concert Hall,Construction & Landscaping,Cosmetics Shop,Curling Ice,Dance Studio
72,North York,4,Park,Japanese Restaurant,Pub,Sushi Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
79,North York,4,Park,Construction & Landscaping,Trail,Bakery,Yoga Studio,Deli / Bodega,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
